In [3]:
import os 

os.environ["PATH"] += os.pathsep + r'C:\Users\user\Downloads\ffmpeg\bin'


In [4]:
import shutil
print(shutil.which("ffmpeg"))


C:\Users\user\Downloads\ffmpeg\bin\ffmpeg.EXE


In [9]:
!pip uninstall -y numpy
!pip install numpy==2.3.0


Found existing installation: numpy 2.3.5
Uninstalling numpy-2.3.5:
  Successfully uninstalled numpy-2.3.5
   ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.0 MB 2.1 MB/s eta 0:00:06
   ---- ----------------------------------- 1.3/13.0 MB 3.4 MB/s eta 0:00:04
   ------ --------------------------------- 2.1/13.0 MB 3.4 MB/s eta 0:00:04
   -------- ------------------------------- 2.9/13.0 MB 3.4 MB/s eta 0:00:03
   ---------- ----------------------------- 3.4/13.0 MB 3.4 MB/s eta 0:00:03
   ------------ --------------------------- 4.2/13.0 MB 3.4 MB/s eta 0:00:03
   -------------- ------------------------- 4.7/13.0 MB 3.5 MB/s eta 0:00:03
   ---------------- ----------------------- 5.5/13.0 MB 3.4 MB/s eta 0:00:03
   ------------------- -------------------- 6.3/13.0 MB 3.4 MB/s eta 0:00:02
   --------------------- ------------------ 7.1/13.0 MB 3.5 MB/s eta 0:00:02
   ------------------------ --------------- 8.1/13.0 M

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.5.0 requires fsspec[http]<=2025.10.0,>=2023.1.0, but you have fsspec 2026.2.0 which is incompatible.


In [1]:
import librosa

sample = "./audio/lsy_audio_2023_58s.mp3"
audio, sr = librosa.load(sample, sr=16000)

print("loaded:", len(audio), "sr:", sr)


loaded: 928000 sr: 16000


In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import librosa

# 1️⃣ device 정리
device = 0 if torch.cuda.is_available() else -1
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

# 2️⃣ 모델 로드
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True,
).to("cuda" if torch.cuda.is_available() else "cpu")

processor = AutoProcessor.from_pretrained(model_id)

# 3️⃣ pipeline 생성 (chunk 옵션 제거)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=device,
    return_timestamps=True,
)

# 4️⃣ ❗ mp3를 파일 경로로 넘기지 말고, 파형으로 로드
sample = "./audio/lsy_audio_2023_58s.mp3"
audio, sr = librosa.load(sample, sr=16000)

# 5️⃣ 파형(array)로 추론
result = pipe({"array": audio, "sampling_rate": sr})
print(result)


Device set to use cpu


{'text': ' 안녕하세요. 이 강의는 GPT API로 챗봇 만들기라는 내용을 다루는 강의입니다. GPT API에 대해서 생소하신 분들도 있을 텐데 우리가 잘 알고 있는 챗GPT, 챗GPT 기능을 이용해서 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요. 그래서 이런 강의들이 사실 많이 있습니다. 그래서 여러 가지들이 있는데 이 강의의 특징이라고 한다면 GPT로 명확한 미션을 달성하는 챗봇 프로그램을 만드는 게 사실 쉽지는 않은데 이걸 어떻게 해서 구현을 하는지 그리고 그게 왜 필요한지에 대해서 좀 이야기를 할 거고요. 그 예제로, 예제는 여러 가지가 될 수 있는데 여기서 예제로 하는 것은 음악 플레이리스트 동영상을 자동으로 대화를 통해서 생성하는 프로그램을 만드는 것을 다루려고 합니다. 그래서 프로그램이 실행되는 모습을 한번 보여드릴게요. 우리가 만들 프로그램은 이런 식으로 이제 나타나게 되고', 'chunks': [{'timestamp': (0.0, 6.3), 'text': ' 안녕하세요. 이 강의는 GPT API로 챗봇 만들기라는 내용을 다루는 강의입니다.'}, {'timestamp': (7.2, 10.38), 'text': ' GPT API에 대해서 생소하신 분들도 있을 텐데'}, {'timestamp': (10.38, 16.72), 'text': ' 우리가 잘 알고 있는 챗GPT, 챗GPT 기능을 이용해서'}, {'timestamp': (16.72, 21.4), 'text': ' 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요.'}, {'timestamp': (22.1, 23.88), 'text': ' 그래서 이런 강의들이 사실 많이 있습니다.'}, {'timestamp': (23.88, 27.48), 'text': ' 그래서 여러 가지들이 있는데 이 강의의 특징이라고 한다면'}, {'timestamp': (27.48, 33.06), 'text': ' GPT로 명확한 미션을 달성하는 챗봇 프로그램을 만드는 게 사

In [7]:
start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("lsy_audio_2023_58s.csv", index=False, sep="|")
display(df)

,start,end,text
0,0.00,6.30,안녕하세요. 이 강의는 GPT API로 챗봇 만들기라는 내용을 다루는 강의입니다.
1,7.20,10.38,GPT API에 대해서 생소하신 분들도 있을 텐데
2,10.38,16.72,"우리가 잘 알고 있는 챗GPT, 챗GPT 기능을 이용해서"
3,16.72,21.40,우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요.
4,22.10,23.88,그래서 이런 강의들이 사실 많이 있습니다.
5,23.88,27.48,그래서 여러 가지들이 있는데 이 강의의 특징이라고 한다면
6,27.48,33.06,GPT로 명확한 미션을 달성하는 챗봇 프로그램을 만드는 게 사실 쉽지는 않은데
7,33.06,37.38,이걸 어떻게 해서 구현을 하는지 그리고 그게 왜 필요한지에 대해서 좀 이야기를 할...
8,38.48,41.48,"그 예제로, 예제는 여러 가지가 될 수 있는데"
9,41.48,48.44,여기서 예제로 하는 것은 음악 플레이리스트 동영상을 자동으로 대화를 통해서 생성하...
